In [1]:
# Installing dependencies
import numpy as np
import pandas as pd
from dLoader import DataLoader, BuySell

In [2]:
def softmax(x):
    # Calculate softmax of x
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [22]:
rdata = DataLoader('AMD').get_data('2018-01-01', '2019-12-31')

In [23]:
# Checking for the maximum mean gain for N shift in day
# When buying near or at close and selling at N days high
for s in range(1, 15):
    days = str(s) + 'B' if s != 0 else 'B'
    # Using Pandas groupby and pd.Grouper to get periodic data
    # Shift -1 to avoid the current day high being the max high
    high = rdata['High'].shift(-1).groupby(by=pd.Grouper(freq=days)).max()
    close = rdata['Close'].groupby(by=pd.Grouper(freq=days)).first()
    hct = high / close - 1
    p = (hct > 0).sum() / len(hct)
    print('Shifted {:3} | Num Trades {:5} | '
          '% Gains {:6.2f}% | % Good Gain {:6.2f}%'.format(
                s, len(hct), hct.mean() * 100, p * 100))

Shifted   1 | Num Trades   521 | % Gains   2.48% | % Good Gain  86.18%
Shifted   2 | Num Trades   261 | % Gains   3.84% | % Good Gain  92.34%
Shifted   3 | Num Trades   174 | % Gains   4.86% | % Good Gain  97.13%
Shifted   4 | Num Trades   131 | % Gains   5.91% | % Good Gain  93.13%
Shifted   5 | Num Trades   105 | % Gains   6.57% | % Good Gain  97.14%
Shifted   6 | Num Trades    87 | % Gains   7.24% | % Good Gain  97.70%
Shifted   7 | Num Trades    75 | % Gains   8.38% | % Good Gain  94.67%
Shifted   8 | Num Trades    66 | % Gains   9.48% | % Good Gain  95.45%
Shifted   9 | Num Trades    58 | % Gains  10.53% | % Good Gain 100.00%
Shifted  10 | Num Trades    53 | % Gains  10.03% | % Good Gain  98.11%
Shifted  11 | Num Trades    48 | % Gains  10.98% | % Good Gain  97.92%
Shifted  12 | Num Trades    44 | % Gains  10.99% | % Good Gain 100.00%
Shifted  13 | Num Trades    41 | % Gains  12.52% | % Good Gain  97.56%
Shifted  14 | Num Trades    38 | % Gains  13.27% | % Good Gain  94.74%
